This notebook explores the valid range of application of `analy_err_in_fit_damp_sine` 

Damping rate

In [ ]:
# TODO: plot the error predicted by analy_err_in_fit_damp_sine vs the damping time. Compare to numerical experiments.